# EE4375-2022: Fifth Lab Session: One-Dimensional Galerkin Finite Element Method using Distributed Memory Parallel Computing

Solves the Poisson equation $- \frac{d^2 \, u(x)}{dx^2} = f(x)$ on the unit bar domain $x \in \Omega=(0,1)$ supplied with various boundary conditions and various source terms. The Galerkin finite element method is employed. Here we target a parallel computing (using distributed memory) imnplementation. 

This problem can be solved using [GridapDistributed](https://gridap.github.io/GridapDistributed.jl/dev/) as students in the bachelor minor Computational Science and Engineering have convincingly shown. It remains valuable to dig for the details. 

General info on [parallel computing in Julia](https://juliaparallel.org/resources/) and [MPI.jl](https://github.com/JuliaParallel/MPI.jl). 

## Import Packages

In [2]:
using LinearAlgebra
using Plots
using LaTeXStrings
using SparseArrays
using BenchmarkTools
using SparseArrays
using Distributed
using SharedArrays

## Section 1: Preprocessing 
- <b> shared memory approach </b>: monolytic in memory approach: assuming global mesh, matrix and rhs vector to be available on all processors: assembly process similar to all processors; 
- <b> distributed memory approach </b>: distribute memory approach: distribute memory approach: distributed assembly and solve; include figure here; 

## Section 2: Shared Memory Assembly of Matrix and Right-Hand Side Vector 
- <b> decompose mesh elements over the processors </b>: observe mesh to be decomposed according to elements and <b>not</b> according to nodes; 
- <b> use shared memory for-loop over elements </b>: the for-loop over elements can run in parallel in shared memory using either the macro @distributed or pmap function from the  [distributed computing section](https://docs.julialang.org/en/v1/stdlib/Distributed/) of the standard library; 
- <b> used SharedVector and SharedMatrix to store answers</b>: using SharedVector and SharedMatrix using [shared arrays](https://docs.julialang.org/en/v1/stdlib/SharedArrays/); each processor fills its part of the matrix; 
- not sure whether [Distributed Arrays](https://juliaparallel.org/DistributedArrays.jl/stable/) offers any advantages; 

<div>
<img src="./figures/dag-fem-1d.jpg" width=400 /> 
<center> Figure 1: Directed acyclic graph representation of decomposed mesh of 4 elements on the interval. A finite element discretization is assumed. </center>   
</div>

In [3]:
# start REPL using julia -p 5 
addprocs(4)
no_procs = nprocs()

#..construct the mesh: see before 
@everywhere N = 100; 
@everywhere h = 1/N; 
@everywhere x = Vector(0:h:1); 

#..Mesh with points and edges 
#..point holds the coordinates of the left and right node of the element
#..edges holds the global indices of the left and right node of the element
@everywhere points = collect( [x[i], x[i+1]] for i in 1:length(x)-1) 
@everywhere edges = collect( [i, i+1] for i in 1:length(x)-1); 

#..Set the source function 
@everywhere fsource(x) = x*(x-1); 

#..Initialize global matrix and right-hand side value 
@everywhere A = spzeros(length(x), length(x)); 
@everywhere f = zeros(length(x), 1); 

#..Perform loop over elements and assemble global matrix and vector 
@distributed for i=1:length(edges) 

  xl, xr = points[i,:][1]
  floc = (xr-xl) * [fsource(xl) fsource(xr)];
  Aloc = (1/(xr-xl))*[1 -1; -1 1]; 

  for j=1:2 
    f[edges[i][j]] += floc[j];
    for k =1:2 
      A[edges[i][j], edges[i][k]] += Aloc[j,k]; 
    end 
  end 

end

LoadError: On worker 2:
UndefVarError: spzeros not defined
Stacktrace:
 [1] top-level scope
[90m   @ [39m[90m[4mnone:1[24m[39m
 [2] [0m[1meval[22m
[90m   @ [39m[90m./[39m[90m[4mboot.jl:368[24m[39m
 [3] [0m[1m#invokelatest#2[22m
[90m   @ [39m[90m./[39m[90m[4messentials.jl:729[24m[39m
 [4] [0m[1minvokelatest[22m
[90m   @ [39m[90m./[39m[90m[4messentials.jl:726[24m[39m
 [5] [0m[1m#114[22m
[90m   @ [39m[90m/Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Distributed/src/[39m[90m[4mprocess_messages.jl:301[24m[39m
 [6] [0m[1mrun_work_thunk[22m
[90m   @ [39m[90m/Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Distributed/src/[39m[90m[4mprocess_messages.jl:70[24m[39m
 [7] [0m[1mrun_work_thunk[22m
[90m   @ [39m[90m/Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Distributed/src/[39m[90m[4mprocess_messages.jl:79[24m[39m
 [8] [0m[1m#100[22m


In [13]:
nprocs()

10

## Section 3: Shared Memory Assembly of Matrix and Right-Hand Side Vector 
- using [PartionedMatrices](https://github.com/fverdugo/PartitionedArrays.jl); need to study the example described at [example](https://www.francescverdugo.com/PartitionedArrays.jl/dev/usage/).   

## Section 4: Shared Memory Linear System Solve 
- <b> first approach </b>: use backslash (nothing to be done, need to check solvers for sharedArrays); 
- using sequential preconditioned [conjugate gradient method](https://en.wikipedia.org/wiki/Conjugate_gradient_method) from [IterativeSolvers.jl]([https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl)
- <b> second approach </b>: use preconditioned conjugate gradient method; using parallel BLAS1 and BLAS2 functions; using [sparse-matrix multiplication](https://github.com/JuliaInv/ParSpMatVec.jl);
- use PCG with proper [overlap of computation and communication](https://netlib.org/linalg/html_templates/node107.html#SECTION00941100000000000000);  

### Parallel BLAS-1 Operations 
Using [ParallelUtilities](https://docs.juliahub.com/ParallelUtilities/SO4iL/0.7.0/). 

#### Vector Norm 
Given vector ${\mathbf v}$, compute its squared norm $\| \mathbf v \|^2$, using pmapsum(x->x^2,v). Alternatively, this is the function dot with the same input argument repeated.  

#### Vectors Inner Product 
Given vectors ${\mathbf v}$ and ${\mathbf w}$, compute their inner product ${\mathbf v} \cdot {\mathbf w}$, using pmapsum((x,y)->x*y,v,w). See [this post](https://discourse.julialang.org/t/function-pmap-multi-argument/74153) for an example of pmap using two arguments. Alternatively, this is the function dot. 
 
#### Vector Update 
Given vectors ${\mathbf v}$ and ${\mathbf w}$, and given a number $\alpha$ (typically the result of an inner product evaluation), compute the vector update ${\mathbf w} = {\mathbf w} + \alpha {\mathbf v}$, using pmapsum((x,y,a)->y+a*x,v,w,alpha). Alternatively, we can update the components of the vector ${\mathbf w}$ elementwise. 

### Parallel BLAS-2 Operations 

#### Sparse Matrix-Vector Multiplication 
Use [ParSpMatVec](https://github.com/JuliaInv/ParSpMatVec.jl/blob/master/test/test_A_mul_B.jl) for sparse matrix vector multiplication. Alternatively, this is the function mul!. 
Background information is provided in e.g. [Parallel Linear Algebra by Deprez](https://fdesprez.github.io/teaching/par-comput/lectures/slides/L8-AlLinPar-2p.pdf); 

### Steepest Descent Method 
Combine ingredients above to implement the steepest descent method as described in Section [Solution to a Linear System](https://en.wikipedia.org/wiki/Gradient_descent#Solution_of_a_linear_system). 

### Conjugate Gradient Method 
Extend steepest descent to the conjugate gradient method as described in Section [Resulting Algorithm](https://en.wikipedia.org/wiki/Conjugate_gradient_method#The_resulting_algorithm). An explanation of the CG algorithm using iterates in explained on [iterative-methods-done-right](https://lostella.github.io/2018/07/25/iterative-methods-done-right.html). 

## Section 5: Distributed Memory Linear System Solve 
Using the PETSc library (as done by the bachelor students). 

## Section 6: Postprocessing 
Visualize the computed solution. 